In [78]:
import os
import json
import requests
from dotenv import load_dotenv

load_dotenv()

True

In [83]:
import pandas as pd

In [79]:
api_key = os.getenv("API-KEY")
headers = {"Authorization":
           f"Bearer {api_key}"}

In [80]:
r = requests.get("https://lambdaschool.instructure.com/api/v1/courses/495/assignments", headers=headers)

In [81]:
r

<Response [200]>

In [82]:
raw = r.json()

with open("m-m.json", "w") as outfile:
    json.dump(raw, outfile)

In [84]:
m_m_df = pd.read_json("m-m.json")

In [85]:
m_m_df.head()

,id,description,due_at,unlock_at,lock_at,points_possible,grading_type,assignment_group_id,grading_standard_id,created_at,...,quiz_id,anonymous_submissions,published,unpublishable,only_visible_to_overrides,locked_for_user,submissions_download_url,post_manually,anonymize_students,require_lockdown_browser
0,8088,,2020-10-30 06:59:59+00:00,NaT,2020-11-21 07:59:59+00:00,1,points,2467,NaN,2020-10-22 18:05:48+00:00,...,6788,False,True,False,False,False,https://lambdaschool.instructure.com/courses/4...,False,False,False
1,8176,,2020-11-06 07:59:00+00:00,NaT,2020-11-21 07:59:59+00:00,1,points,2467,NaN,2020-10-22 18:05:57+00:00,...,6876,False,True,False,False,False,https://lambdaschool.instructure.com/courses/4...,False,False,False
2,8020,,2020-11-13 07:59:00+00:00,NaT,2020-11-21 07:59:59+00:00,1,points,2467,NaN,2020-10-22 18:05:43+00:00,...,6721,False,True,False,False,False,https://lambdaschool.instructure.com/courses/4...,False,False,False
3,8080,,2020-10-27 06:59:59+00:00,NaT,2020-11-21 07:59:59+00:00,1,points,2467,NaN,2020-10-22 18:05:48+00:00,...,6780,False,True,False,False,False,https://lambdaschool.instructure.com/courses/4...,False,False,False
4,7887,,2020-10-28 06:59:59+00:00,NaT,2020-11-21 07:59:59+00:00,1,points,2467,NaN,2020-10-22 18:05:36+00:00,...,6617,False,True,False,False,False,https://lambdaschool.instructure.com/courses/4...,False,False,False
